# Fairify Artifact
Artifact of the paper "Fairify: Fairness Verification of Neural Networks" from ICSE 2023.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [1]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [2]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 14 compute_cascadelake_r nodes ready to use


### Reserve node

In [3]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [4]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,bdffed27-9530-4651-89f6-d6605a9df5af
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.2.183 (v4) Type: fixed MAC: bc:97:e1:c4:30:80
Network Name,sharednet1
Created At,2025-03-28T12:46:04Z
Keypair,trovi-ef7b87a
Reservation Id,163ce65d-e28d-4959-a9a0-4d761d34b2f7
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [5]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.233 port 22.


Connection successful


## Run Fairify

Now, we can finally run Fairify. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [8]:
my_server.execute("rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git")

Cloning into 'Fairify'...


<Result cmd='rm -rf Fairify && git clone https://github.com/radhofan/Fairify.git' exited=0>

In [ ]:
my_server.execute("chmod +x Fairify/reproduce.sh")
my_server.execute("bash Fairify/reproduce.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.

Running `shell init`, which:
 - modifies RC file: "/home/cc/.bashrc"
 - generates config for root prefix: "/home/cc/miniconda"
 - sets mamba executable to: "/home/cc/miniconda/bin/mamba"
The following has been added in your "/home/cc/.bashrc" file

# >>> mamba initialize >>>
# !! Contents within this block are managed by 'mamba shell init' !!
export MAMBA_EXE='/home/cc/miniconda/bin/mamba';
export MAMBA_ROOT_PREFIX='/home/cc/miniconda';
__mamba_setup="$("$MAMBA_EXE" shell hook --shell bash --root-prefix "$MAMBA_ROOT_PREFIX" 2> /dev/null)"
if [ $? -eq 0 ]; then
    eval "$__mamba_setup"
else
    alias mamba="$MAMBA_EXE"  # Fallback on help from mamba activate
fi
unset __mamba_setup
# <<< mamba initialize <<<



warning  libmamba 'repo.anaconda.com', a commercial channel hosted by Anaconda.com, is used.
    
warning  libmamba Please make sure you understand Anaconda Terms of Services.
    
warning  libmamba See: https://legal.anaconda.com/policies/en/




Transaction

  Prefix: /home/cc/miniconda/envs/fairify

  Updating specs:

   - python=3.9


  Package               Version  Build           Channel         Size
───────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────

  + _libgcc_mutex           0.1  main            pkgs/main     Cached
  + _openmp_mutex           5.1  1_gnu           pkgs/main     Cached
  + ca-certificates   2025.2.25  h06a4308_0      pkgs/main     Cached
  + ld_impl_linux-64       2.40  h12ee557_0      pkgs/main     Cached
  + libffi                3.4.4  h6a678d5_1      pkgs/main     Cached
  + libgcc-ng            11.2.0  h1234567_1      pkgs/main     Cached
  + libgomp              11.2.0  h1234567_1      pkgs/main     Cached
  + libstdcxx-ng         11.2.0  h1234567_1      pkgs/main     Cached
  + ncurses                 6.4  h6a678d5_0      pkgs/main     Cached
  + openssl              3.0.16  h5eee18b_0      

2025-03-28 14:34:33.902430: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-28 14:34:33.902448: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


...

', '1', '0', '0', '1', '1', '1', '0', '0', '0']
[-0.10133696] [0.09153414]
[-0.10133749] [0.09153396]
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
unknown
Pruning done!
3.92 % HEURISTIC PRUNING
76.47 % TOTAL PRUNING
Verifying ...
unknown
V time:  100.028
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  70.559
['0', '0', '0', '0', '6599', '0', '0', '3', '0', '3', '1', '0', '0', '0', '1', '0', '1', '0', '0', '0']
['0', '0', '0', '0', '6599', '0', '0', '3', '0', '3', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0']
[-0.12619555] [0.06626326]
[-0.12619555] [0.06626326]
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  10.726
['0', '12', '1', '9', '18046', '0', '0', '4', '2', '1', '0', '1', '0', '1', '2', '3', '1', '0', '1', '0']
['0', '12', '1', '9', '18046', '0', '0', '4', '2', '1', '0', '0', '0', '1', '2', '3', '1', '0', '1', '0']



Processing Models:  80%|████████  | 4/5 [1:08:10<17:54, 1074.24s/it]

******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  16.009
['2', '79', '2', '3', '16700', '2', '0', '2', '1', '4', '0', '0', '1', '1', '1', '0', '1', '0', '1', '1']
['2', '79', '2', '3', '16700', '2', '0', '2', '1', '4', '0', '1', '1', '1', '1', '0', '1', '0', '1', '1']
[-0.0288679] [0.16345084]
[-0.02886784] [0.16345102]
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  13.325
['2', '73', '1', '3', '7800', '1', '1', '1', '0', '3', '0', '0', '1', '1', '3', '0', '2', '1', '0', '0']
['2', '73', '1', '3', '7800', '1', '1', '1', '0', '3', '0', '1', '1', '1', '3', '0', '2', '1', '0', '0']
[-0.00045162] [0.19186744]
[-0.00045139] [0.19186765]
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  6.968
['0', '0', '0', '0', '9199', '0', '0', '1', '0', '4', '2', '0', '0', '0', '1', '1', '1', '0', '0', '0']
['0', '0', '0', '0', '9199', '0'


Processing Partitions:   0%|          | 0/201 [00:00<?, ?it/s]

, '0', '1', '0', '4', '2', '1', '0', '0', '1', '1', '1', '0', '0', '0']
[-0.18351626] [0.00905782]
[-0.1835162] [0.00905794]
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
sat
V time:  3.632
['1', '31', '0', '9', '12499', '0', '0', '4', '1', '1', '2', '1', '1', '0', '4', '3', '2', '0', '1', '0']
['1', '31', '0', '9', '12499', '0', '0', '4', '1', '1', '2', '0', '1', '0', '4', '3', '2', '0', '1', '0']
[0.00205439] [-0.19026434]
[0.00205427] [-0.19026452]
******************
INTERVAL BASED PRUNING
SINGULAR VERIFICATION
Pruning done!
Verifying ...
unknown
Pruning done!
1.96 % HEURISTIC PRUNING
74.51 % TOTAL PRUNING
Verifying ...
unknown
V time:  100.042
******************
==================  COMPLETED MODEL GC-1.h5
==================  STARTING MODEL GC-2.h5
###################
